In [73]:
#Загрузка библиотек
#%pip install lightgbm
#%pip install catboost
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)  
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import  accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as  lgb
from xgboost.sklearn import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.preprocessing import StandardScaler, LabelBinarizer
# auxiliary function
from sklearn.preprocessing import LabelEncoder
def random_colors(number_of_colors):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
    return color


import warnings
warnings.filterwarnings('ignore')

In [311]:
#Загрузка данных
df= pd.read_csv('C:/Users/Maria/Desktop/olimp/train.csv')
test_final= pd.read_csv('C:/Users/Maria/Desktop/olimp/test.csv')

In [258]:
#Смотрим на размерность данных
print(test_final.shape, df.shape)

(133, 11) (309, 12)


train_y = pd.DataFrame(train['Onshore/Offshore'])
train_X = train.drop(['Onshore/Offshore'], axis=1)
train_X= pd.get_dummies(train_X)
test_X = test
train_y= pd.get_dummies(train_y)

In [312]:
x = df.drop(['Onshore/Offshore'], axis=1)
y = (df['Onshore/Offshore'])

In [296]:
#Смотрим на таргет переменную 
y.value_counts()

ONSHORE             211
OFFSHORE             93
ONSHORE-OFFSHORE      5
Name: Onshore/Offshore, dtype: int64

In [313]:
#Применяем label encoder, так как наша переменная многофакторная 
encoder = LabelEncoder()
y = pd.DataFrame(encoder.fit_transform(y.values))

In [281]:
#В датасете данные распределены неравномерно, данные класса 2 практически отсутсвуют
y[0].value_counts()

1    211
0     93
2      5
Name: 0, dtype: int64

In [314]:
#избавляемся от категориальных переменных
temp = pd.get_dummies(pd.concat([x,test_final],keys=[0,1]))
#temp = pd.get_dummies(pd.concat([Xtrain,Xtest],keys=[0,1]), columns=['x'])
x,test_final = temp.xs(0),temp.xs(1)

In [315]:
print(test_final.shape,x.shape)

(133, 183) (309, 183)


In [316]:
encoder = LabelEncoder()
y = pd.DataFrame(encoder.fit_transform(y.values))

In [317]:
#Нормируем переменные для усреднениях их весов в предсказании
scaler = preprocessing.StandardScaler().fit(x)
x= pd.DataFrame(scaler.fit_transform(x), columns = x.columns)

In [318]:
x

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EROSION/GRAVITY/EVAPORITE/EXTENSION,...,Lithology_DOLOMITIC LIMESTONE,Lithology_GLAUCONITIC SANDSTONE,Lithology_LIMESTONE,Lithology_LOW-RESISTIVITY SANDSTONE,Lithology_SANDSTONE,Lithology_SHALE,Lithology_SHALY SANDSTONE,Lithology_SILTSTONE,Lithology_THINLY-BEDDED SANDSTONE,Lithology_VOLCANICS
0,-0.915915,1.252099,0.055619,0.271259,-0.426453,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
1,0.925775,-0.279905,-0.155141,0.140235,-0.293839,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,2.521338,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
2,0.563836,-0.530090,-0.664477,-0.776938,-0.451956,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,2.521338,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
3,-0.469133,-0.354509,0.336632,-0.645913,0.139705,2.080285,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
4,-1.627224,-0.405751,-0.601249,0.795358,-0.286699,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1.867902,0.528673,0.319069,1.450481,1.057798,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,5.773503,-1.181249,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
305,-1.448112,-0.478094,-0.372926,1.843555,7.178421,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,5.773503,-1.181249,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
306,0.995478,-0.354509,0.031031,-0.645913,-0.471542,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713
307,2.752039,0.136818,1.004039,-0.252839,-0.462157,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.14072,-0.080713


In [319]:
#smote позволяет уравновесить классы
x,y = smote.fit_sample(x, y)

In [320]:
x.shape

(515, 183)

In [321]:
#делим тренировочную выборку на тест и трен.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 200)

In [53]:
##%pip install imbalanced-learn
#from imblearn import under_sampling, over_sampling
#from imblearn.over_sampling import SMOTE


In [54]:
#smote= SMOTE('minority',k_neighbors=2)
#x_train, y_train= smote.fit_sample(x_train, y_train)
#print(x_train.shape)
#y_train

(354, 164)


array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 2, 1, 0,
       0, 1, 1, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [304]:
#Пробуем разные модели,начиная с Логистической регрессии
lr_model = LogisticRegression()
lr_model.fit(x_train,y_train)
lr_predict = lr_model.predict(x_test)

print('Logistic Regression - ',accuracy_score(lr_predict,y_test))




Logistic Regression -  0.8838709677419355


In [229]:
#Счиатаем метрики качества
import sklearn.metrics as metrics

mae = metrics.mean_absolute_error(y_test, lr_predict)
mse = metrics.mean_squared_error(y_test, lr_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,lr_predict)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

Results of sklearn.metrics:
MAE: 0.11612903225806452
MSE: 0.11612903225806452
RMSE: 0.3407771005482389
R-Squared: 0.7874447661130581


In [362]:
from sklearn.metrics import classification_report
report = classification_report(y_test,lr_predict)
print(report) 

              precision    recall  f1-score   support

           0       0.69      0.71      0.70        28
           1       0.86      0.84      0.85        61
           2       0.99      1.00      0.99        66

    accuracy                           0.88       155
   macro avg       0.85      0.85      0.85       155
weighted avg       0.88      0.88      0.88       155



In [363]:
#Тренируем SVM модель
svm_model = SVC(kernel='linear')
svm_model.fit(x_train,y_train)
svc_predict = svm_model.predict(x_test)

print('SVM - ',accuracy_score(svc_predict,y_test))

SVM -  0.8838709677419355


In [366]:
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}

In [367]:
#Поиск гиперпараметров
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

In [368]:
grid.fit(x,y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.1s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.1s
[CV] C=1, gamma=1, kernel=linear .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.1s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.1s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.1s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.2s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.2s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.3s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.3s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] .

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:   32.3s finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['linear', 'rbf']},
             verbose=2)

In [369]:
grid.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [ ]:
predicted = grid.predict(x_test)
print(classification_report(y_test,predicted))

In [371]:
test_final

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EROSION/GRAVITY/EVAPORITE/EXTENSION,...,Lithology_DOLOMITIC LIMESTONE,Lithology_GLAUCONITIC SANDSTONE,Lithology_LIMESTONE,Lithology_LOW-RESISTIVITY SANDSTONE,Lithology_SANDSTONE,Lithology_SHALE,Lithology_SHALY SANDSTONE,Lithology_SILTSTONE,Lithology_THINLY-BEDDED SANDSTONE,Lithology_VOLCANICS
0,-1.271570,-0.356769,-0.611787,-0.645913,-0.472317,2.080285,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
1,-1.778627,-0.571536,-0.653939,1.319456,-0.227533,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
2,-0.336584,-0.450965,-0.646914,-0.645913,-0.464911,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
3,0.963769,5.577580,0.196126,0.009210,-0.370347,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
4,0.578119,-0.496180,-0.471280,-0.776938,-0.344845,-0.480703,3.231787,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,-0.344583,-0.126931,0.666823,1.057407,0.547746,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
129,-1.492961,-0.526322,-0.429128,1.319456,-0.023512,-0.480703,3.231787,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
130,1.249435,-0.450965,-0.190267,0.271259,-0.395850,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,7.106335,-0.080713
131,-0.796221,-0.292716,-0.646914,1.319456,0.547746,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713


In [364]:
mae = metrics.mean_absolute_error(y_test, svc_predict)
mse = metrics.mean_squared_error(y_test, svc_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,svc_predict)
report = classification_report(y_test,svc_predict)


In [365]:
mae = metrics.mean_absolute_error(y_test, svc_predict)
mse = metrics.mean_squared_error(y_test, svc_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,svc_predict)

print(report) 
print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

              precision    recall  f1-score   support

           0       0.68      0.68      0.68        28
           1       0.85      0.85      0.85        61
           2       1.00      1.00      1.00        66

    accuracy                           0.88       155
   macro avg       0.84      0.84      0.84       155
weighted avg       0.88      0.88      0.88       155

Results of sklearn.metrics:
MAE: 0.11612903225806452
MSE: 0.11612903225806452
RMSE: 0.3407771005482389
R-Squared: 0.7874447661130581


In [234]:
#Пробуем модель Naive Bayes
nb_model = GaussianNB()
nb_model.fit(x_train,y_train)
nb_predict = nb_model.predict(x_test)

print('Naive bayes - ',accuracy_score(nb_predict,y_test))


mae = metrics.mean_absolute_error(y_test, nb_predict)
mse = metrics.mean_squared_error(y_test, nb_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,nb_predict)
report = classification_report(y_test,nb_predict)
print(report) 
print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

Naive bayes -  0.8193548387096774
              precision    recall  f1-score   support

           0       0.54      0.79      0.64        28
           1       0.87      0.64      0.74        61
           2       0.96      1.00      0.98        66

    accuracy                           0.82       155
   macro avg       0.79      0.81      0.78       155
weighted avg       0.85      0.82      0.82       155

Results of sklearn.metrics:
MAE: 0.18064516129032257
MSE: 0.18064516129032257
RMSE: 0.4250237185032414
R-Squared: 0.6693585250647571


In [235]:
#Деревья решений
dt_model = DecisionTreeClassifier(max_leaf_nodes=3)
dt_model.fit(x_train,y_train)
dt_predict = dt_model.predict(x_test)

mae = metrics.mean_absolute_error(y_test, dt_predict)
mse = metrics.mean_squared_error(y_test, dt_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,dt_predict)
report = classification_report(y_test,dt_predict)

print('Decision Tree - ',accuracy_score(dt_predict,y_test))

report = classification_report(y_test,dt_predict)
print(report) 
print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

Decision Tree -  0.7677419354838709
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.73      0.89      0.80        61
           2       0.80      0.98      0.88        66

    accuracy                           0.77       155
   macro avg       0.51      0.62      0.56       155
weighted avg       0.63      0.77      0.69       155

Results of sklearn.metrics:
MAE: 0.2903225806451613
MSE: 0.4064516129032258
RMSE: 0.637535577754862
R-Squared: 0.2560566813957035


In [236]:
#Случайный лес
rfc_model = RandomForestClassifier(max_depth=3)
rfc_model.fit(x_train,y_train)
rfc_predict = rfc_model.predict(x_test)


mae = metrics.mean_absolute_error(y_test, rfc_predict)
mse = metrics.mean_squared_error(y_test, rfc_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,rfc_predict)
report = classification_report(y_test,rfc_predict)


print('Random Forest - ',accuracy_score(rfc_predict,y_test))

report = classification_report(y_test,rfc_predict)
print(report) 
print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

Random Forest -  0.8064516129032258
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        28
           1       0.67      0.98      0.80        61
           2       0.98      0.98      0.98        66

    accuracy                           0.81       155
   macro avg       0.55      0.66      0.59       155
weighted avg       0.68      0.81      0.73       155

Results of sklearn.metrics:
MAE: 0.1935483870967742
MSE: 0.1935483870967742
RMSE: 0.43994134506405985
R-Squared: 0.645741276855097


In [381]:
#Extra Trees
etc_model = ExtraTreesClassifier(n_estimators=320, n_jobs=4, min_samples_split=25,
                            min_samples_leaf=2, max_depth= 32)
etc_model.fit(x,y)
etc_predict = etc_model.predict(x_test)


ae = metrics.mean_absolute_error(y_test, etc_predict)
mse = metrics.mean_squared_error(y_test, etc_predict)
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(y_test,etc_predict)
report = classification_report(y_test,etc_predict)


print('Extra Tree Classifier - ',accuracy_score(etc_predict,y_test))

report = classification_report(y_test,etc_predict)
print(report) 
print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

Extra Tree Classifier -  0.9612903225806452
              precision    recall  f1-score   support

           0       0.96      0.82      0.88        28
           1       0.92      0.98      0.95        61
           2       1.00      1.00      1.00        66

    accuracy                           0.96       155
   macro avg       0.96      0.94      0.95       155
weighted avg       0.96      0.96      0.96       155

Results of sklearn.metrics:
MAE: 0.11612903225806452
MSE: 0.03870967741935484
RMSE: 0.19674775073518588
R-Squared: 0.9291482553710194


In [361]:
#Градиентный бустинг
xg_model= XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.26, max_delta_step=0, max_depth=3,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


xg_model = xg_model.fit(x_train,y_train)
xg_model.score(x_test, y_test)
xg_model = xg_model.predict(test_final)
xg_model
xg_model= pd.DataFrame(xg_model)
xg_model.to_csv(r'C:\Users\Maria\Desktop\xg_model3.csv', index = False, header=False)

In [ ]:
xg_model= XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
xg_model= XGBClassifier(learning_rate= 0.26 , max_depth=3, n_estimators= 1000, 'seed': 42)

xg_model = xg_model.fit(x_train,y_train)
xg_model.score(x_test, y_test)
xg_model = xg_model.predict(test_final)
xg_model
xg_model= pd.DataFrame(xg_model)
xg_model.to_csv(r'C:\Users\Maria\Desktop\xg_model2.csv', index = False, header=False)

In [354]:
test_final= pd.DataFrame(scaler.transform(test_final), columns = test_final.columns)
test_final

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EROSION/GRAVITY/EVAPORITE/EXTENSION,...,Lithology_DOLOMITIC LIMESTONE,Lithology_GLAUCONITIC SANDSTONE,Lithology_LIMESTONE,Lithology_LOW-RESISTIVITY SANDSTONE,Lithology_SANDSTONE,Lithology_SHALE,Lithology_SHALY SANDSTONE,Lithology_SILTSTONE,Lithology_THINLY-BEDDED SANDSTONE,Lithology_VOLCANICS
0,-1.271570,-0.356769,-0.611787,-0.645913,-0.472317,2.080285,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
1,-1.778627,-0.571536,-0.653939,1.319456,-0.227533,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
2,-0.336584,-0.450965,-0.646914,-0.645913,-0.464911,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
3,0.963769,5.577580,0.196126,0.009210,-0.370347,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
4,0.578119,-0.496180,-0.471280,-0.776938,-0.344845,-0.480703,3.231787,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,-0.344583,-0.126931,0.666823,1.057407,0.547746,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
129,-1.492961,-0.526322,-0.429128,1.319456,-0.023512,-0.480703,3.231787,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
130,1.249435,-0.450965,-0.190267,0.271259,-0.395850,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,7.106335,-0.080713
131,-0.796221,-0.292716,-0.646914,1.319456,0.547746,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713


In [353]:
xg_model= XGBClassifier(learning_rate= 0.26 , max_depth=3, n_estimators= 1000, seed=  42)

xg_model = xg_model.fit(x_train,y_train)
xg_model.score(x_test, y_test)
xg_model = xg_model.predict(test_final)
xg_model
#xg_model= pd.DataFrame(xg_model)
#xg_model.to_csv(r'C:\Users\Maria\Desktop\xg_model2.csv', index = False, header=False)

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EROSION/GRAVITY/EVAPORITE/EXTENSION,...,Lithology_DOLOMITIC LIMESTONE,Lithology_GLAUCONITIC SANDSTONE,Lithology_LIMESTONE,Lithology_LOW-RESISTIVITY SANDSTONE,Lithology_SANDSTONE,Lithology_SHALE,Lithology_SHALY SANDSTONE,Lithology_SILTSTONE,Lithology_THINLY-BEDDED SANDSTONE,Lithology_VOLCANICS
0,2275,325.0,30.0,13.0,0.04,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,500,40.0,18.0,28.0,240.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,5548,200.0,20.0,13.0,7.30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10100,8200.0,260.0,18.0,100.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,8750,140.0,70.0,12.0,125.00,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,5520,630.0,394.0,26.0,1000.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
129,1500,100.0,82.0,28.0,440.00,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
130,11100,200.0,150.0,20.0,75.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
131,3939,410.0,20.0,28.0,1000.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [343]:
#Поиск гиперпараметров для градиентного бустинга 
param_grid = {'learning_rate': [0.05, 0.1, 0.26, 0.3], #so called `eta` value
              'max_depth': [3, 6, 8],
              'n_estimators': [1000], #number of trees, change it to 1000 for better rw32aesults
              'missing':[-999],
              'seed': [42]}

searchCV = GridSearchCV(estimator= xg_model, scoring='r2',  param_grid=param_grid, verbose=True)

# WARNING: This could take some time to run.
searchCV.fit(x, y)

print('Best index:', searchCV.best_index_)
print('Best score:', searchCV.best_score_)
print('Best params:', searchCV.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 12.4min finished


Best index: 6
Best score: 0.7762752988614496
Best params: {'learning_rate': 0.26, 'max_depth': 3, 'missing': -999, 'n_estimators': 1000, 'seed': 42}


In [342]:
xg_model.fit(
    x_train, 
    y_train, 
    eval_metric="merror", 
    eval_set=[(x_train, y_train), (x_test, y_test)])

[0]	validation_0-merror:0.03056	validation_1-merror:0.14194
[1]	validation_0-merror:0.02222	validation_1-merror:0.14194
[2]	validation_0-merror:0.01944	validation_1-merror:0.12903
[3]	validation_0-merror:0.02500	validation_1-merror:0.14839
[4]	validation_0-merror:0.01944	validation_1-merror:0.13548
[5]	validation_0-merror:0.01111	validation_1-merror:0.12903
[6]	validation_0-merror:0.00833	validation_1-merror:0.12903
[7]	validation_0-merror:0.00556	validation_1-merror:0.12258
[8]	validation_0-merror:0.00556	validation_1-merror:0.11613
[9]	validation_0-merror:0.00556	validation_1-merror:0.11613
[10]	validation_0-merror:0.00278	validation_1-merror:0.11613
[11]	validation_0-merror:0.00278	validation_1-merror:0.11613
[12]	validation_0-merror:0.00278	validation_1-merror:0.11613
[13]	validation_0-merror:0.00278	validation_1-merror:0.10968
[14]	validation_0-merror:0.00278	validation_1-merror:0.10968
[15]	validation_0-merror:0.00278	validation_1-merror:0.10323
[16]	validation_0-merror:0.00000	v

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [376]:
#Поиск гиперпараметров для Extra Tree Classification
from sklearn import pipeline
from sklearn.pipeline import Pipeline
etc_model = ExtraTreesClassifier( n_estimators=320, n_jobs=4, min_samples_split=25,
                            min_samples_leaf=2, max_depth= 32)


In [92]:
ExtraTreesClassifier().get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [375]:
from sklearn.calibration import CalibratedClassifierCV
calibrated_forest = CalibratedClassifierCV( base_estimator=ExtraTreesClassifier(n_estimators=10))
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
cv = StratifiedKFold(n_splits=3, random_state=0)

param_grid = { 
    "n_estimators": [320, 340, 360, 380, 400, 1000],
    "max_depth": [25, 30, 32, 34, 38, 45, 10],
    "min_samples_leaf": [2,10,15,20,30,35]
}
searchCV = GridSearchCV(estimator= model, scoring='accuracy', cv=cv, param_grid=param_grid, verbose=True)
searchCV.fit(x, y)

print('Best index:', searchCV.best_index_)
print('Best score:', searchCV.best_score_)
print('Best params:', searchCV.best_params_)


Fitting 3 folds for each of 252 candidates, totalling 756 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 756 out of 756 | elapsed: 40.4min finished


Best index: 72
Best score: 0.89900947459087
Best params: {'max_depth': 32, 'min_samples_leaf': 2, 'n_estimators': 320}


In [241]:
test_final

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EROSION/GRAVITY/EVAPORITE/EXTENSION,...,Lithology_DOLOMITIC LIMESTONE,Lithology_GLAUCONITIC SANDSTONE,Lithology_LIMESTONE,Lithology_LOW-RESISTIVITY SANDSTONE,Lithology_SANDSTONE,Lithology_SHALE,Lithology_SHALY SANDSTONE,Lithology_SILTSTONE,Lithology_THINLY-BEDDED SANDSTONE,Lithology_VOLCANICS
0,2275,325.0,30.0,13.0,0.04,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,500,40.0,18.0,28.0,240.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,5548,200.0,20.0,13.0,7.30,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10100,8200.0,260.0,18.0,100.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,8750,140.0,70.0,12.0,125.00,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,5520,630.0,394.0,26.0,1000.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
129,1500,100.0,82.0,28.0,440.00,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
130,11100,200.0,150.0,20.0,75.00,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
131,3939,410.0,20.0,28.0,1000.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [377]:
test_final= pd.DataFrame(scaler.transform(test_final), columns = test_final.columns)
test_final

,Depth,Gross,Netpay,Porosity,Permeability,Tectonic regime_COMPRESSION,Tectonic regime_COMPRESSION/EROSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION,Tectonic regime_COMPRESSION/EROSION/EXTENSION/LINKED,Tectonic regime_COMPRESSION/EROSION/GRAVITY/EVAPORITE/EXTENSION,...,Lithology_DOLOMITIC LIMESTONE,Lithology_GLAUCONITIC SANDSTONE,Lithology_LIMESTONE,Lithology_LOW-RESISTIVITY SANDSTONE,Lithology_SANDSTONE,Lithology_SHALE,Lithology_SHALY SANDSTONE,Lithology_SILTSTONE,Lithology_THINLY-BEDDED SANDSTONE,Lithology_VOLCANICS
0,-1.271570,-0.356769,-0.611787,-0.645913,-0.472317,2.080285,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
1,-1.778627,-0.571536,-0.653939,1.319456,-0.227533,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
2,-0.336584,-0.450965,-0.646914,-0.645913,-0.464911,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
3,0.963769,5.577580,0.196126,0.009210,-0.370347,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
4,0.578119,-0.496180,-0.471280,-0.776938,-0.344845,-0.480703,3.231787,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,-0.344583,-0.126931,0.666823,1.057407,0.547746,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
129,-1.492961,-0.526322,-0.429128,1.319456,-0.023512,-0.480703,3.231787,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713
130,1.249435,-0.450965,-0.190267,0.271259,-0.395850,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,-1.181249,-0.05698,-0.099015,-0.099015,7.106335,-0.080713
131,-0.796221,-0.292716,-0.646914,1.319456,0.547746,-0.480703,-0.309426,-0.05698,-0.05698,-0.080713,...,-0.080713,0.0,-0.396615,-0.173205,0.846562,-0.05698,-0.099015,-0.099015,-0.140720,-0.080713


In [382]:
etc_predict_final = etc_model.predict(test_final)
etc_predict_final= pd.DataFrame(etc_predict_final)

In [383]:
#Сохраняем результаты лучшей модели
etc_predict_final.to_csv(r'C:\Users\Maria\Desktop\ETC_pred.csv', index = False, header=False)